# Installations

In [41]:
!pip install transformers accelerate datasets sentencepiece scikit-learn
!pip install torch torchvision torchaudio

  Using cached scikit_learn-1.5.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp311-cp311-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp311-cp311-macosx_12_0_arm64.whl (11.0 MB)
Using cached scipy-1.14.1-cp311-cp311-macosx_14_0_arm64.whl (23.1 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [1]:
import gc
import torch

import pandas as pd

from typing import List, Dict, Union

from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import T5TokenizerFast
from transformers import T5ForConditionalGeneration
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments

/Users/khushpatel2002/inno/pmldl_project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load dataset

In [1]:

df = pd.read_csv("hf://datasets/s-nlp/paradetox/train.tsv", sep="\t")
df

/Users/khushpatel2002/inno/pmldl_project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,en_toxic_comment,en_neutral_comment
0,he had steel balls too !,he was brave too!
1,"dude should have been taken to api , he would ...",It would have been good if he went to api. He ...
2,"im not gonna sell the fucking picture , i just...","I'm not gonna sell the picture, i just want to..."
3,the garbage that is being created by cnn and o...,the news that is being created by cnn and othe...
4,the reason they dont exist is because neither ...,The reason they don't exist is because neither...
...,...,...
19739,when they do shit like this .,when they do stuff like this
19740,"but if saying "" fuck that group "" is much more...","but if saying"" that group is bad"" is much more..."
19741,"it hurts how judgemental assholes view them , ...",It hurts how judgemental that people view them...
19742,shit we probably literally blow that up in a w...,We probably litteralky blow that up in a week.


In [3]:
xydf = pd.DataFrame({'source': df["en_toxic_comment"].values.tolist(), 'target': df["en_neutral_comment"].values.tolist()})

In [4]:
xydf

,source,target
0,he had steel balls too !,he was brave too!
1,"dude should have been taken to api , he would ...",It would have been good if he went to api. He ...
2,"im not gonna sell the fucking picture , i just...","I'm not gonna sell the picture, i just want to..."
3,the garbage that is being created by cnn and o...,the news that is being created by cnn and othe...
4,the reason they dont exist is because neither ...,The reason they don't exist is because neither...
...,...,...
19739,when they do shit like this .,when they do stuff like this
19740,"but if saying "" fuck that group "" is much more...","but if saying"" that group is bad"" is much more..."
19741,"it hurts how judgemental assholes view them , ...",It hurts how judgemental that people view them...
19742,shit we probably literally blow that up in a w...,We probably litteralky blow that up in a week.


# Model checkpoints

In [5]:
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
model_name = "t5-small"
tokenizer = T5TokenizerFast.from_pretrained(model_name)
df_train, df_test = train_test_split(xydf, test_size=300)
print(df_train.shape[0], df_test.shape[0])


x1 = tokenizer(df_train.source.tolist(), truncation=True)
y1 = tokenizer(df_train.target.tolist(), truncation=True)
x2 = tokenizer(df_test.source.tolist(), truncation=True)
y2 = tokenizer(df_test.target.tolist(), truncation=True)

19444 300


In [7]:
class PairsDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        assert idx < len(self.x['input_ids'])
        item = {key: val[idx] for key, val in self.x.items()}
        item['decoder_attention_mask'] = self.y['attention_mask'][idx]
        item['labels'] = self.y['input_ids'][idx]
        return item
    
    @property
    def n(self):
        return len(self.x['input_ids'])

    def __len__(self):
        return self.n # * 2
    
train_dataset = PairsDataset(x1, y1)
test_dataset = PairsDataset(x2, y2)
len(train_dataset), len(test_dataset)

(19444, 300)

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=4, drop_last=True, shuffle=True, num_workers=1)
test_dataloader = DataLoader(test_dataset, batch_size=4, drop_last=True, shuffle=True, num_workers=1)

In [11]:

class DataCollatorWithPadding:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.tokenizer.pad(
            features,
            padding=True,
        )
        ybatch = self.tokenizer.pad(
            {'input_ids': batch['labels'], 'attention_mask': batch['decoder_attention_mask']},
            padding=True,
        ) 
        batch['labels'] = ybatch['input_ids']
        batch['decoder_attention_mask'] = ybatch['attention_mask']
        
        return {k: torch.tensor(v) for k, v in batch.items()}

In [12]:
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [14]:


if torch.backends.mps.is_available():
    device = torch.device('mps')  # Use MPS (Metal Performance Shaders) on Apple Silicon
else:
    device = torch.device('cpu')  # Fallback to CPU if MPS is not available

model.to(device)


print(device)


mps


In [16]:
save_name = 'models/t5-baseline'

In [20]:

training_args = TrainingArguments(
    output_dir=save_name,             # output directory
    overwrite_output_dir=True,
    num_train_epochs=3,               # total number of training epochs
    per_device_train_batch_size=4,    # batch size per device during training
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,     # batch size for evaluation
    warmup_steps=300,                 # number of warmup steps for learning rate scheduler
    weight_decay=0,                   # strength of weight decay
    learning_rate=3e-5,
    logging_dir='./logs',             # directory for storing logs
    logging_steps=100,
    eval_steps=100,
    evaluation_strategy='steps',      # evaluation strategy (e.g., 'steps', 'epoch')
    save_total_limit=1,               # limit the number of saved checkpoints
    save_steps=5000,                  # save checkpoints every 5000 steps
)


In [21]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:
trainer = Trainer(
    model=model,                         
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [23]:
gc.collect()
torch.cuda.empty_cache() 

In [24]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  3%|▎         | 100/3645 [00:57<24:23,  2.42it/s]

{'loss': 4.9263, 'grad_norm': 36.59382629394531, 'learning_rate': 9.999999999999999e-06, 'epoch': 0.08}


                                                  
  3%|▎         | 100/3645 [01:01<24:23,  2.42it/s]

{'eval_loss': 3.6027963161468506, 'eval_runtime': 4.3918, 'eval_samples_per_second': 68.309, 'eval_steps_per_second': 8.653, 'epoch': 0.08}


  5%|▌         | 200/3645 [01:46<23:19,  2.46it/s]  

...

                                                   
 99%|█████████▉| 3600/3645 [26:40<00:17,  2.52it/s]

{'eval_loss': 0.5829160809516907, 'eval_runtime': 1.5051, 'eval_samples_per_second': 199.323, 'eval_steps_per_second': 25.248, 'epoch': 2.96}


100%|██████████| 3645/3645 [26:59<00:00,  2.25it/s]

{'train_runtime': 1620.0174, 'train_samples_per_second': 36.007, 'train_steps_per_second': 2.25, 'train_loss': 1.0346248799390754, 'epoch': 3.0}


TrainOutput(global_step=3645, training_loss=1.0346248799390754, metrics={'train_runtime': 1620.0174, 'train_samples_per_second': 36.007, 'train_steps_per_second': 2.25, 'total_flos': 399857836228608.0, 'train_loss': 1.0346248799390754, 'epoch': 2.999382843036412})

In [25]:
trainer.evaluate()

100%|██████████| 38/38 [00:01<00:00, 21.86it/s]


{'eval_loss': 0.5828280448913574,
 'eval_runtime': 1.8275,
 'eval_samples_per_second': 164.163,
 'eval_steps_per_second': 20.794,
 'epoch': 2.999382843036412}

In [26]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [31]:
inputs = tokenizer('The internal policy of the fucking Trump is stupid.', return_tensors='pt')
inputs = {k: v.to(device) for k, v in inputs.items()}
for t in model.generate(**inputs, num_return_sequences=10, do_sample=False, num_beams=10):
    print(tokenizer.decode(t, skip_special_tokens=True))

The internal policy of Trump is stupid.
The internal policy of the Trump is stupid.
The internal policy of Trump is stupid
The internal policy of Trump is not good.
The internal policy of the Trump is stupid
The internal policy of Trump is bad.
The internal policy of the Trump is not good.
The internal policy of the Trump is bad.
The internal policy of Trump is not good
The internal policy of the Trump is not good
